(np-utils:stimscan)=
# Stimulation parameter scan

This utility allows you to automatically search for optimal stimulation parameters on a given MEA.

```{note}
Neuroplatform access (and booking for shared access) is required to use this utility.
```


## Installation

To install all dependencies, run the following command:

```bash
pip install git+https://github.com/FinalSpark-np/np-utils.git#egg=np_utils[SSN]
```

## Usage

### Imports

Import the needed classes :


In [ ]:
from neuroplatform import Experiment, StimPolarity
from np_utils.stim_scan import StimParamGrid, StimScan, MEAType

### Defining a StimParamGrid

In [ ]:
grid = StimParamGrid(
    amplitudes=[1, 2, 3],
    durations=[100, 200],
    polarities=[StimPolarity.NegativeFirst, StimPolarity.PositiveFirst],
    mea_type=MEAType.MEA4x8,  # OR MEAType.MEA32
)
print(grid.total_combinations())
grid.display_grid()

### Running the scan
````{margin}
```{seealso}
See [Experiment and token](core:exp-and-tokens) for relevant API usage.
```
````	

Next, define which channels to run the scan on (in absolute index, e.g. for MEA2, channels 32-64)
Provide your experiment, the grid of parameters, and the delays as well as number of repeats per channel.

In [ ]:
EXPERIMENT = Experiment("...")  # use your token here
scan = StimScan(
    fs_experiment=EXPERIMENT,
    parameter_grid=grid,
    delay_btw_stim=1,  # delay between stimulations
    delay_btw_channels=5,  # delay between channels
    repeats_per_channel=5,  # number of repeats per channel for a single parameter set
    scan_channels=range(
        32, 64
    ),  # channels to scan, here MEA2. Must match electrodes allowed by your token
)

```{caution}
A progress bar for the scan will be displayed.<br>
**Take care that the duration of the scan does not exceed that of your booking.**
You can reduce the number of parameters, the number of repeats, or the number of channels to scan to reduce the duration.
Shortening the delay between stimulations also works, but may cause fatigue and affect the quality of the results.
```

In [ ]:
scan.run()

### Displaying results

#### Showing all stimulations

To access the list of all stimulations and associated parameters, use :

In [ ]:
stim_df = scan.get_stimulation_parameter_history()
stim_df

Finally, to show all events after stimulation :

In [ ]:
scan.plot_all_stims(
    s_before=1, s_after=2.5
)  # arguments define how long to show before the first stim and after the last respectively

You may also use `scan.plot_all_stims_for_param` and `scan.plot_all_stims_for_channel` to visualize the stimulation events specifically for a given parameter or channel.

#### Additional plotting options

##### Selecting specific channels

The `show_electrodes` argument lets you pick which channels to display. By default, all channels are displayed.

##### Event interval visualization

Finally, you can use the `guideline_freq`argument to show lines every `guideline_freq` seconds. This can be useful to visualize the stimulation events in relation to the stimulation frequency.

To do this, define the `guideline_freq` argument as the desired period in seconds as such : "{n_sec}s" (e.g. "1s" for 1 second).

This is None by default, which will not display any guidelines.

## Future additions

Summary metrics as well as best parameter automated selection may be added in a future version.
